In [1]:
import pandas as pd
import requests
from datetime import timedelta, date
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows',200)

In [2]:
# start = date(2017, 6, 1)
# end = date(2018, 6, 1)


# df = pd.DataFrame()

# for sdate in range((end - start).days):
#     fdate = (start + timedelta(sdate)).strftime('%y%m%d')
#     url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_" + fdate + ".txt"
#     try:
#         data = pd.read_csv(url)
#         data['file'] = fdate
#         df = df.append(data)
#     except:
#         pass

In [3]:
# df['date'] = pd.to_datetime(df.DATE)

In [4]:
# df['turnid'] = df['STATION'] + '/' + df['SCP'] + '/' + df['C/A']

In [5]:
# # Remove readings at unusual times (that don't land on an even hour)
# df = df[df.TIME.str.contains(':00:00', regex=False)]


# df = df.sort_values(['STATION','turnid','TIME'])


# df = df.rename(columns={ df.columns[10]: "EXITS" })

In [6]:
# pickle.dump(df, open('turnstiledata','wb'))

In [7]:
df = pickle.load(open('turnstiledata','rb'))

In [8]:
df['STATION']= df['STATION'] +'/' +df['LINENAME'] 

In [9]:
conditions = [  df.TIME.isin(['23:00:00','00:00:00','01:00:00','02:00:00']),
                df.TIME.isin(['03:00:00','04:00:00','05:00:00','06:00:00']),
                df.TIME.isin(['07:00:00','08:00:00','09:00:00','10:00:00']),
                df.TIME.isin(['11:00:00','12:00:00','13:00:00','14:00:00']),
                df.TIME.isin(['15:00:00','16:00:00','17:00:00','18:00:00']),
                df.TIME.isin(['19:00:00','20:00:00','21:00:00','22:00:00'])]

choices = ['00:00','04:00','08:00','12:00','16:00','20:00']

df['TIME'] = np.select(conditions, choices, default=df.TIME)

In [10]:
df = df.groupby(['STATION','turnid','date','TIME'], as_index=False).sum()

df['dayofweek'] = df.date.dt.day_name()



In [11]:
df['entries'] = df.ENTRIES.shift(-1) - df.ENTRIES
df['exits'] = df.EXITS.shift(-1) - df.EXITS
df['passages'] = df.entries + df.exits



In [13]:
df['weekend/weekday'] = np.where(df['dayofweek'].isin( ['Saturday','Sunday']), 'Weekend','Weekday')

In [14]:
df_weekend= df[df['weekend/weekday']=='Weekend']
df_weekday= df[df['weekend/weekday']!='Weekend']

In [17]:
df_weekend

STATION                   turnid       date   TIME  \
0    0             1 AV/L       1 AV/00-00-00/H007 2017-05-27  00:00   
     1             1 AV/L       1 AV/00-00-00/H007 2017-05-27  04:00   
     2             1 AV/L       1 AV/00-00-00/H007 2017-05-27  08:00   
     3             1 AV/L       1 AV/00-00-00/H007 2017-05-27  12:00   
     4             1 AV/L       1 AV/00-00-00/H007 2017-05-27  16:00   
     5             1 AV/L       1 AV/00-00-00/H007 2017-05-27  20:00   
     6             1 AV/L       1 AV/00-00-00/H007 2017-05-28  00:00   
     7             1 AV/L       1 AV/00-00-00/H007 2017-05-28  04:00   
     8             1 AV/L       1 AV/00-00-00/H007 2017-05-28  08:00   
     9             1 AV/L       1 AV/00-00-00/H007 2017-05-28  12:00   
     10            1 AV/L       1 AV/00-00-00/H007 2017-05-28  16:00   
     11            1 AV/L       1 AV/00-00-00/H007 2017-05-28  20:00   
     42            1 AV/L       1 AV/00-00-00/H007 2017-06-03  00:00   
     43            1 AV/L       1 AV/00-00-00/H007 2017-06-03  04:00   
     44            1 AV/L       1 AV/00-00-00/H007 2017-06-03  08:00   
     45            1 AV/L       1 AV/00-00-00/H007 2017-06-03  12:00   
     46            1 AV/L       1 AV/00-00-00/H007 2017-06-03  16:00   
     47            1 AV/L       1 AV/00-00-00/H007 2017-06-03  20:00   
     48            1 AV/L       1 AV/00-00-00/H007 2017-06-04  00:00   
     49            1 AV/L       1 AV/00-00-00/H007 2017-06-04  04:00   
     50            1 AV/L       1 AV/00-00-00/H007 2017-06-04  08:00   
     51            1 AV/L       1 AV/00-00-00/H007 2017-06-04  12:00   
     52            1 AV/L       1 AV/00-00-00/H007 2017-06-04  16:00   
     53            1 AV/L       1 AV/00-00-00/H007 2017-06-04  20:00   
     84            1 AV/L       1 AV/00-00-00/H007 2017-06-10  00:00   
     85            1 AV/L       1 AV/00-00-00/H007 2017-06-10  04:00   
     86            1 AV/L       1 AV/00-00-00/H007 2017-06-10  08:00   
     87            1 AV/L       1 AV/00-00-00/H007 2017-06-10  12:00   
     88            1 AV/L       1 AV/00-00-00/H007 2017-06-10  16:00   
     89            1 AV/L       1 AV/00-00-00/H007 2017-06-10  20:00   
     90            1 AV/L       1 AV/00-00-00/H007 2017-06-11  00:00   
     91            1 AV/L       1 AV/00-00-00/H007 2017-06-11  04:00   
     92            1 AV/L       1 AV/00-00-00/H007 2017-06-11  08:00   
     93            1 AV/L       1 AV/00-00-00/H007 2017-06-11  12:00   
     94            1 AV/L       1 AV/00-00-00/H007 2017-06-11  16:00   
     95            1 AV/L       1 AV/00-00-00/H007 2017-06-11  20:00   
     126           1 AV/L       1 AV/00-00-00/H007 2017-06-17  00:00   
     127           1 AV/L       1 AV/00-00-00/H007 2017-06-17  04:00   
     128           1 AV/L       1 AV/00-00-00/H007 2017-06-17  08:00   
     129           1 AV/L       1 AV/00-00-00/H007 2017-06-17  12:00   
     130           1 AV/L       1 AV/00-00-00/H007 2017-06-17  16:00   
     131           1 AV/L       1 AV/00-00-00/H007 2017-06-17  20:00   
     132           1 AV/L       1 AV/00-00-00/H007 2017-06-18  00:00   
     133           1 AV/L       1 AV/00-00-00/H007 2017-06-18  04:00   
     134           1 AV/L       1 AV/00-00-00/H007 2017-06-18  08:00   
     135           1 AV/L       1 AV/00-00-00/H007 2017-06-18  12:00   
     136           1 AV/L       1 AV/00-00-00/H007 2017-06-18  16:00   
     137           1 AV/L       1 AV/00-00-00/H007 2017-06-18  20:00   
     168           1 AV/L       1 AV/00-00-00/H007 2017-06-24  00:00   
     169           1 AV/L       1 AV/00-00-00/H007 2017-06-24  04:00   
     170           1 AV/L       1 AV/00-00-00/H007 2017-06-24  08:00   
     171           1 AV/L       1 AV/00-00-00/H007 2017-06-24  12:00   
     172           1 AV/L       1 AV/00-00-00/H007 2017-06-24  16:00   
     173           1 AV/L       1 AV/00-00-00/H007 2017-06-24  20:00   
     174           1 AV/L       1 AV/00-00-00/H007 2017-06

In [18]:
df_weekend_median = df_weekend.groupby(['STATION','TIME'], as_index=False).median()

df_weekend_count = df_weekend.groupby(['STATION','TIME'], as_index=False).count()
df_weekend_median['count'] = df_weekend_count.iloc[:,4]

df_weekend_median = df_weekend_median.drop(columns =['ENTRIES','EXITS'])

sorted_df_weekend=df_weekend_median[df_weekend_median['count']>4].sort_values(by='passages', ascending = False)



In [19]:
sorted_df_weekend

,STATION,TIME,entries,exits,passages,count
1210,BEDFORD AV/L,16:00,436.0,634.0,1270.0,1350
147,14 ST-UNION SQ/456LNQRW,12:00,489.5,522.5,1191.0,1228
1209,BEDFORD AV/L,12:00,384.5,594.5,1165.0,1348
148,14 ST-UNION SQ/456LNQRW,16:00,559.0,445.5,1162.5,1216
153,14 ST-UNION SQ/LNQR456W,12:00,431.0,481.0,1057.5,2692
878,8 AV/N,08:00,486.5,468.0,1024.0,450
879,8 AV/N,12:00,558.0,429.0,1004.5,450
2528,SPRING ST/6,16:00,352.0,304.0,988.5,620
154,14 ST-UNION SQ/LNQR456W,16:00,456.5,334.0,970.5,2692
1211,BEDFORD AV/L,20:00,457.0,461.0,943.0,1351


In [21]:
df_weekday_median = df_weekday.groupby(['STATION','TIME'], as_index=False).median()

df_weekday_count = df_weekday.groupby(['STATION','TIME'], as_index=False).count()
df_weekday_median['count'] = df_weekday_count.iloc[:,4]

df_weekday_median = df_weekday_median.drop(columns =['ENTRIES','EXITS'])
df_weekday_median

,STATION,TIME,entries,exits,passages,count
0,1 AV/L,00:00,11.0,9.5,23.0,2594
1,1 AV/L,04:00,77.0,184.0,316.0,2598
2,1 AV/L,08:00,339.0,517.0,846.0,2594
3,1 AV/L,12:00,278.0,303.0,786.0,2595
4,1 AV/L,16:00,436.0,332.0,1540.0,2599
5,1 AV/L,20:00,523.5,471.0,1026.0,2584
6,103 ST-CORONA/7,00:00,13.0,41.0,78.0,2340
7,103 ST-CORONA/7,04:00,634.0,32.0,717.0,2340
8,103 ST-CORONA/7,08:00,634.5,95.0,806.0,2328
9,103 ST-CORONA/7,12:00,290.0,201.0,564.0,2335


In [22]:
sorted_df_weekday=df_weekday_median[df_weekday_median['count']>15].sort_values(by='passages', ascending = False)
sorted_df_weekday

,STATION,TIME,entries,exits,passages,count
424,23 ST/FM,16:00,1340.0,503.0,1868.0,2584
148,14 ST-UNION SQ/456LNQRW,16:00,911.0,608.0,1852.0,3067
2778,YORK ST/F,16:00,1546.5,115.5,1726.5,780
154,14 ST-UNION SQ/LNQR456W,16:00,1113.0,425.0,1701.0,6725
1210,BEDFORD AV/L,16:00,365.5,1108.0,1589.0,3380
422,23 ST/FM,08:00,292.0,896.5,1580.0,2584
4,1 AV/L,16:00,436.0,332.0,1540.0,2599
586,42 ST-BRYANT PK/BDFM7,16:00,1084.0,286.0,1511.0,5459
2622,TIMES SQ-42 ST/ACENQRS1237W,16:00,948.0,603.0,1503.0,2079
147,14 ST-UNION SQ/456LNQRW,12:00,737.0,544.0,1501.0,3067


In [ ]:
# build tourist index

In [69]:
weekday_8am = df_weekday_median[df_weekday_median.TIME == '08:00'].sort_values(by='STATION')
weekday_8am = weekday_8am[weekday_8am.passages >0]
weekday_8am


,STATION,TIME,entries,exits,passages,count
2,1 AV/L,08:00,339.0,517.0,846.0,2594
8,103 ST-CORONA/7,08:00,634.5,95.0,806.0,2328
14,103 ST/1,08:00,540.0,159.0,688.0,1559
20,103 ST/6,08:00,484.0,434.0,854.0,1547
26,103 ST/BC,08:00,414.5,161.5,569.5,780
32,104 ST/A,08:00,129.0,7.0,133.0,1296
38,104 ST/JZ,08:00,55.0,0.0,51.5,1032
44,110 ST/6,08:00,369.5,283.0,705.0,1560
50,111 ST/7,08:00,682.0,120.0,838.5,1298
56,111 ST/A,08:00,82.0,7.0,90.0,1820


In [64]:
weekend_noon =df_weekend_median[df_weekend_median.TIME =='12:00'].sort_values(by='STATION')
weekend_noon =weekend_noon.drop(columns=['TIME','entries','exits','count'])
weekend_noon =weekend_noon[weekend_noon.passages>0]
weekend_noon

,STATION,passages
3,1 AV/L,722.0
9,103 ST-CORONA/7,518.5
15,103 ST/1,527.0
21,103 ST/6,611.0
27,103 ST/BC,386.0
33,104 ST/A,38.0
39,104 ST/JZ,6.0
45,110 ST/6,501.0
51,111 ST/7,526.0
57,111 ST/A,44.0


In [70]:
tourist_index = weekday_8am[:]

In [71]:
tourist_index = tourist_index.merge(weekend_noon, how= 'inner', left_on='STATION', right_on='STATION', suffixes= ['_weekday','_weekend'])

In [72]:
tourist_index['tourist'] = tourist_index.passages_weekend/tourist_index.passages_weekday

In [73]:
tourist_index.sort_values(by='tourist')

,STATION,TIME,entries,exits,passages_weekday,count,passages_weekend,tourist
178,AVENUE N/F,08:00,46.0,3.0,55.0,1738,1.0,0.018182
71,231 ST/1,08:00,18.0,10.0,92.0,2593,7.0,0.076087
271,EASTCHSTER/DYRE/5,08:00,168.0,94.0,265.0,1298,23.0,0.086792
332,KINGS HWY/F,08:00,161.0,72.0,262.0,1785,26.5,0.101145
6,104 ST/JZ,08:00,55.0,0.0,51.5,1032,6.0,0.116505
276,EXCHANGE PLACE/1,08:00,869.0,56.0,925.0,1,117.0,0.126486
102,47-50 STS ROCK/BDFM,08:00,77.0,1062.0,1192.0,9342,173.0,0.145134
368,NEWARK HW BMEBE/1,08:00,176.5,16.5,193.0,2,37.0,0.191710
340,LEXINGTON AV/53/EM6,08:00,80.0,575.0,708.0,8247,139.0,0.196328
83,33 ST-RAWSON ST/7,08:00,119.0,382.0,539.5,2596,109.5,0.202966
